Q2. Counting the pairs with k different from an integer list
eg: list = [1,3,5] and k = 2
expected: we will have 2 pairs: {(1,3), (3,5)}
Note: we also consider the negative numbers. 

sol.<br />
Brute force appraoch is to run through 2 for loops to iterate the list and compare each element with rest of the elements. its time complexity is O(n square)
efficient approach:<br />
Assumption : list is sorted in assending order as per given example. <br />
We use sliding window technique with variable window size. compare each adjecent elements and check for difference. Time complexity is O(n) <br/>
<br/>
if list is not sorted the we need to sort it out first and proceed with below function, which will increase the time complexity to O(nlogn)

In [74]:

def getDiffPair_Sorted(arr,n): 
    #taking distinct elements
    arr = list(set(arr))
    
    size = len(arr) 
    #result set with tuples
    res = set()
  
    # Initialize indices
    i,j = 0,1
  
    # Search pairs sequentially
    while i < size and j < size: 
  
        if i != j and arr[j]-arr[i] == n: 
            res.add((arr[i],arr[j]))
            i += 1
            j += 1
  
        elif arr[j] - arr[i] < n: 
            j+=1
        
        else: 
            i+=1
    
    return res

In [75]:
getDiffPair_Sorted([-3,-5,-7,-11,-13] ,-2)

{(-5, -7), (-3, -5)}

Q3. Return the list of indices. The indices is a sublist points to the same person. The same persons means they have the same name or email or phone. <br />
eg: data = [ ("username1","phone_number1", "email1"), ("usernameX","phone_number1", "emailX"), ("usernameZ","phone_numberZ", "email1Z"), ("usernameY","phone_numberY", "emailX"), ]  <br />
expected: [[0,1,3][2]]
<br />

In [76]:
import numpy as np
def matchPerson(arr):
    data = np.array(arr)
    res = []
    #compare each element of row with another row and form matrix
    matchPersons = sum([user == data for user in data])

    # person  match with any other person has sum > 3
    match = list(np.where(np.sum(matchPersons, axis=1) > 3)[0])
    res.append(match)
    
    # the person which are not there in match set
    nonMatch = list(set(range(len(data))) ^ set(match))
    res.append(nonMatch)
    
    return res

In [77]:
arr = [ ('username1','phone_number1', 'email1'),
       ('usernameX','phone_number1', 'emailX'),
       ('usernameZ','phone_numberZ', 'email1Z'),
       ('usernameY','phone_numberY', 'emailX') ]
matchPerson(arr)

[[0, 1, 3], [2]]

Q4. Implement the Forward propagation & Backward propagation for a three layers Neural Network. X,W
and b can be random. 

sol. <br/>
Forward Propagation <br/>
Forward Propagation refers to the movement of the input through the hidden layers to the output layers. In forward propagation, the information travels in a single direction FORWARD. The input layer supplies the input to the hidden layers and then the output is generated. There is no backward movement.

Backpropagation <br/>
When we define a neural network, we assign random weights and bias values to our nodes. Once we have received the output for a single iteration, we can calculate the error of the network. This error is then fed back to the network along with the gradient of the cost function to update the weights of the network. These weights are then updated so that the errors in the subsequent iterations are reduced. This updating of weights using the gradient of the cost function is known as back-propagation.
<br/>
<ol>
<li> X : input layer size</li>
<li> h : hidden layer size</li>
<li> W : weights</li>
<li> b : bias</li>
<li> Y : output layer size = (_,1)</li>
<li> Activation functions : Sigmoid(x) = 1/ (1+e-x), derivative of Sigmoid(x) = Sigmoid(x) * (1 - Sigmoid(x))
<li> cost function : means square error = 1/m ∑(y – ^y)2 </li>
<li> optimization algo : gradient descent</li>
</ol>

<ol>
    <li> inputs : X,W,b</li>
    <li> improvements: we can enhance the below implmentation by using double linked lists for each layer. so that we can add number of hidden layers dynamically. while propagations, we can use liked list concept to move from one layer to next layer in both forward and backward directions</li>
</ol>

In [78]:
import numpy as np
class NN:
    def __init__(self, input_layer, hidden_layer, output_layer, W1,W2,B1,B2):
        self.hidden_layer = hidden_layer
        self.output_layer = output_layer
        self.input_layer = input_layer
        self.W1 = W1
        self.W2 = W2
        self.B1 = B1
        self.B2 = B2
        self.error = 0
        
    def sigmoid(self,X):
        return 1 / (1 + np.exp(-X))

    #we are using gradient descent optimization, so we take derivative (slope) of activation
    def sigmoid_der(self,x):
        sig = self.sigmoid(x)
        return sig * (1 - sig)
    
    def mse(self,y_true, y_pred):
        return (np.sum(np.power(y_pred - y_true, 2))) / len(y_true)
        
    def forwardAndBackwardPro(self,x_train,y_train):
        for x, y_true in zip(x_train, y_train):
            #forward propagation
            #input layer to hidden layer operations
            #linear output
            h1_ = np.dot(x, self.W1) + self.B1
            #non-linear output by sigmoid
            h1 = self.sigmoid(h1_)

            #hidden layer to output layer operations
            #linear output
            y_ = np.dot(h1, self.W2) + self.B2
            #non-linear output by sigmoid
            self.y = self.sigmoid(y_)

            # Error computation
            self.error += self.mse(y_true,y_train)
            
            # Backward Propagation
            #initialize new weights and biases
            dW1 = np.zeros((self.input_layer, self.hidden_layer))
            dW2 = np.zeros((self.hidden_layer, self.output_layer))
            dB1 = np.zeros(self.hidden_layer)
            dB2 = np.zeros(self.output_layer)
            
            #calculate bias from output layer to hidden layer
            #bias b'2= (y-ytrue) * f'(y_)
            for k in range(self.output_layer):
                dB2[k] = (self.y[k] - y_true[k]) * self.sigmoid_der(y_[k])
            
            #calculate weight from output layer to hidden layer
            #weight w'2= b'2 * h1
            for j in range(self.hidden_layer):
                for k in range(self.output_layer):
                    dW2[j][k] =  dB2[k] * h1[j]

            #calculate bias from hidden layer to input layer
            #bias b'1= sum (b'2 * w'2 * f'(h1))
            for j in range(self.hidden_layer):
                dB1[j] = sum([dB2[k] * self.W2[j][k] * self.sigmoid_der(h1_[j]) for k in range(self.output_layer)])

            #calculate weight from hidden layer to input layer
            #weight w'1 = b'1 * input
            for i in range(self.input_layer):
                for j in range(self.hidden_layer):
                    dW1[i][j] = dB1[j] * x[i]
            
        return self.y


In [79]:
# Number of neurons for each layer
input_layer = 2
hidden_layer = 4
output_layer = 1
W1 = np.random.rand(input_layer, hidden_layer)
B1 = np.random.rand(hidden_layer)
W2 = np.random.rand(hidden_layer, output_layer)
B2 = np.random.rand(output_layer)
nunet = NN(input_layer,hidden_layer,output_layer,W1,W2,B1,B2)
x_train = np.array([[0,1], [1,1], [1,0], [1,1]])
y_train = np.array([[1], [1], [0], [0]])
nunet.forwardAndBackwardPro(x_train,y_train)

array([0.90683733])

Q1. The random variable X and Y have the following joint probability density <br/>
𝑓𝑋𝑌(𝑥, 𝑦) = 𝑒−𝑥−𝑦 0 < 𝑥 < ∞,0 < 𝑦 < ∞ <br />
            0 𝑒𝑙𝑠𝑒𝑤ℎ𝑒𝑟𝑒
What is 𝑃(𝑋 < 𝑌) ? <br/>

sol. <br/>
Both random variables are continuous as they have joint probability density function. <br />
P(X<Y) = $$\int_{0}^x\int_{x}^y fXY(x,y)dxdy$$  <br/>
$$\int_{0}^x\int_{x}^y 1/e^x  1/e^y dxdy$$  <br/>
= 1/e